In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
from multiprocessing import Pool
import csv

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
browser = webdriver.Chrome()
browser.get('https://www.douyu.com/110')

In [3]:
cast_name = browser.find_element_by_xpath('//a[@class="zb-name"]/h1').text
date = time.strftime('%m-%d')

data_file = 'data_in_csv/%s-%s.csv' % (cast_name, date)
log_file = 'log/%s-%s.txt' % (cast_name, date)

with open(log_file, 'a+', encoding='utf-8') as log:
    log.write(time.strftime('>>>>logging time:%Y/%m/%d-%H:%M:%S<<<<<\n'))
    
while True:
    date = time.strftime('%m-%d')
    data_file = 'data_in_csv/%s-%s.csv' % (cast_name, date)
    log_file = 'log/%s-%s.txt' % (cast_name, date)
    
    s = time.time()
    chunks = re.findall('<li class="jschartli hy-chat(.*?)</li>', browser.page_source, re.S)
    
    node = browser.find_elements_by_xpath('//ul[@class="c-list"]/li[@class="jschartli hy-chat  "]')[-1]
    wait = WebDriverWait(browser, 3600)

    names, levels, fans, dms = [], [], [], []
    for c in chunks:
        levels.append(re.findall('data-level="(.*?)" ', c, re.S)[0])
        if re.findall('<i>(.*?)</i>',c, re.S):
            fans.append(re.findall('<i>(.*?)</i>', c, re.S)[0])
        else:
            fans.append('无粉丝牌')
        names.append(re.findall('data-noble=".*?">(.*?)：<', c, re.S)[0])
        dms.append(re.findall('chatid=.*?">(.*?)</span', c, re.S)[0])
        
    with open(data_file, 'a+', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(zip(names, levels, fans, dms))
        
    count = len(names)  
    
    e = time.time()        
    print('InsertTime: %8.4f' % (e-s), end=' , ')
    print('AddNumber: %8d' % (count), end=' , ')
    s_ = time.time()
    wait.until(EC.staleness_of(node))
    e_ = time.time()
    print('WaitingTime: %8.2f' % (e_-s_), end=' , ')
    t = time.strftime('%c')
    print('DateNTime: %s\n' % (t))
    with open(log_file, 'a+', encoding='utf-8') as log:
        log.write('InsertTime:  %8.4f , ' % (e - s))
        log.write('AddNumber:   %8d , '   % (count))
        log.write('WaitingTime: %8.2f , ' % (e_-s_))
        log.write('DateNTime: %s\n' % (t))

InsertTime:   0.0708 , AddNumber:        9 , WaitingTime:    89.99 , DateNTime: Fri Aug 17 11:29:52 2018

InsertTime:   0.1077 , AddNumber:      161 , WaitingTime:    63.74 , DateNTime: Fri Aug 17 11:30:56 2018

InsertTime:   0.1227 , AddNumber:      179 , WaitingTime:    63.23 , DateNTime: Fri Aug 17 11:31:59 2018

InsertTime:   0.1171 , AddNumber:      178 , WaitingTime:    76.82 , DateNTime: Fri Aug 17 11:33:16 2018

InsertTime:   0.1117 , AddNumber:      168 , WaitingTime:    16.69 , DateNTime: Fri Aug 17 11:33:33 2018

InsertTime:   0.0928 , AddNumber:      191 , WaitingTime:    71.31 , DateNTime: Fri Aug 17 11:34:44 2018

InsertTime:   0.0996 , AddNumber:      155 , WaitingTime:    94.04 , DateNTime: Fri Aug 17 11:36:19 2018

InsertTime:   0.1037 , AddNumber:      164 , WaitingTime:   107.14 , DateNTime: Fri Aug 17 11:38:06 2018

InsertTime:   0.0947 , AddNumber:      170 , WaitingTime:    84.96 , DateNTime: Fri Aug 17 11:39:31 2018

InsertTime:   0.1089 , AddNumber:      174 , W

KeyboardInterrupt: 